In [3]:
import itertools
import numpy as np
import math
from dismal import generator_matrices_ext
from scipy.stats import poisson
from dismal.generator_matrices import GeneratorMatrix
from dismal.likelihood_matrix import LikelihoodMatrix
from scipy import linalg

Define generator matrices

In [4]:
mig_mat1 = np.array(
    [
        [0, 0.1],
        [0.1, 0]
    ]
)


popsizes1 = [1, 1]

np.set_printoptions(precision=3)
q1 = generator_matrices_ext.q(mig_mat1, popsizes1)

In [5]:
mig_mat2 = np.array(
    [
        [0, 0],
        [0, 0]
    ]
)


popsizes2 = [1, 1]

np.set_printoptions(precision=3)
q2 = generator_matrices_ext.q(mig_mat2, popsizes2)

In [6]:
q2

array([[-1.,  0.,  0.,  1.],
       [ 0., -1.,  0.,  1.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.]])

In [7]:
q3 = np.array(
    [[-1, 0, 0, 1],
    [0, -1, 0, 1],
    [0, 0, -1, 1],
    [0, 0, 0, 0]]
)

q3

array([[-1,  0,  0,  1],
       [ 0, -1,  0,  1],
       [ 0,  0, -1,  1],
       [ 0,  0,  0,  0]])

In [8]:
def matrix_multiplication(matrices):
    m = matrices[0]
    for i in range(1, len(matrices)):
        m = m @ matrices[i]
    
    return m

In [9]:
def alpha_matrix(eigenvals, s_vals, t_start, t_end):
    """Results are identical to old implementation"""

    eigenvals = np.array(eigenvals)
    s_vals = np.array(s_vals)

    if t_end is not None:

        alphas = [eigenvals/(eigenvals+1) * (1/(eigenvals+1))**s
              * np.exp(eigenvals * t_start)
              * (poisson.cdf(s, t_start * (eigenvals+1)) 
                 - poisson.cdf(s, t_end * (eigenvals+1))) for s in s_vals] 
    
    else:
        alphas = [eigenvals/(eigenvals+1) * (1/(eigenvals+1))**s
              * np.exp(eigenvals * t_start)
              * poisson.cdf(s, t_start * (eigenvals+1)) for s in s_vals] 
    
    return np.transpose(alphas)

In [10]:
q1, q2 = GeneratorMatrix.from_params([1,1,1,1,1,1,1,0.1,0.1,0.1,0.1])

eigenvectors = [q1.eigenvectors, q2.eigenvectors]
eigenvalues = [-q1.eigenvalues[0:3], -q2.eigenvalues[0:3]]

In [11]:
def generalised_likelihood_2pop(X, lmbda, P, gamma, t_vals, s_vals):

    """X = eigenvectors, lmbda=eigenvalues, P=P matrices. Need to adjust indices for additional populations."""

    n_matrices = len(X)
    assert n_matrices == len(lmbda)
    assert n_matrices == len(P)

    likelihoods = []

    XX = [linalg.inv(x) @ np.diag(x[:, 3]) for x in X]
    
    for i in [0,1,2]:
        most_recent = (XX[0][i, 0:3] 
                       @ alpha_matrix(lmbda[0], s_vals, t_start=0, t_end=t_vals[0]))


        middle = ([P[n-1][i, 0:3] 
                   @ XX[n][0:3, 0:3] 
                   @ alpha_matrix(lmbda[n], s_vals, t_start=t_vals[n-1], t_end=t_vals[n]) 
                   for n in range(1, n_matrices)])
        

        most_ancient = (1 - matrix_multiplication(P)[i,3] 
                        * alpha_matrix(gamma, s_vals, t_start=t_vals[-1], t_end=None))

        
        state_likelihood = most_recent + middle + most_ancient
        likelihoods.append(state_likelihood)
    

    return np.log(likelihoods)
    

In [16]:
p_matrices = [linalg.expm(x) for x in eigenvectors]

In [17]:
generalised_likelihood_2pop(eigenvectors, eigenvalues, p_matrices, gamma=1, t_vals=[1,2], s_vals=np.array([[1,1,1], [1,1,1], [1,1,1]]))

/var/folders/qk/10crn8pd5_d038f0tw_7z2800000gp/T/ipykernel_6837/3226956619.py:32: RuntimeWarning: invalid value encountered in log
  return np.log(likelihoods)


array([[[ 0.383,  0.383,  0.383],
        [ 0.383,  0.383,  0.383],
        [ 0.383,  0.383,  0.383]],

       [[   nan,    nan,    nan],
        [   nan,    nan,    nan],
        [   nan,    nan,    nan]],

       [[-0.786, -0.786, -0.786],
        [-0.786, -0.786, -0.786],
        [-0.786, -0.786, -0.786]]])

Comparison with existing DISMaL implementation

In [105]:
lm = LikelihoodMatrix(params={"theta0":1, "theta1":1, "theta2":1, "theta1_prime":1, "theta2_prime":1, "t1":1, "v":1, "M1_star":0.1, "M2_star":0.1, "M1_prime_star":0.1, "M2_prime_star": 0.1}, S=np.array([[1,1,1], [1,1,1], [1,1,1]]))

In [112]:
np.sum(lm.matrix)

13.980985964077146